In [108]:
from pyscf import gto, scf, lo
import numpy as np
from functools import reduce
from pyscf.lo.orth import pre_orth_ao_atm_scf
import ase
from pyscf import lo

T,F=True,False
np.set_printoptions(precision=2,suppress=True)

def calc(aseobj, bst='cc-pvdz', icab=F):
    zs = aseobj.numbers
    nh = (zs==1).sum()
    coords = aseobj.positions
    assert zs[0]>1 and np.all(zs[1:]==1) #zs = []; coords = []
    spin = sum(zs)%2
    atom = ''
    na = len(aseobj)
    for i in range(na):
        x,y,z = coords[i]
        ai = aseobj[i]
        atom += '%s %.8f %.8f %.8f;'%(ai.symbol, x,y,z)
    if icab:
        basis = {'H':'sto-3g'}
    else:
        basis = {'H':bst}
    for si in ['C','N','O','F']: basis[si] = bst
    mol = gto.M(atom=atom, basis=basis, verbose=0, spin=spin)
    mf = None
    if not icab:
        mf = scf.RHF(mol)
        mf.kernel()
    return mol, mf

def get_hao(mol):
    zs = mol.atom_charges()
    nh = (zs==1).sum()
    s1 = mol.intor_symmetric('int1e_ovlp')
    b1 = pre_orth_ao_atm_scf(mol)
    sb = reduce( np.dot, (b1.conjugate().T, s1, b1) )
    aolbs = mol.ao_labels(); nao = len(aolbs)
    sb_hx = sb[-nh:,:-nh] # overlap matrix H-X
    u,d,vh = np.linalg.svd(sb_hx, full_matrices=False, compute_uv=True)
    a1 = np.dot(vh.T, u.T)
    # now Schmidt orthogonalization
    n1 = nh
    n2 = nao - nh
    t = np.eye(n2)
    t[:,:nh] = a1
    for i in range(nh,n2):
        for j in range(i):
            cj = t[i,j] 
            t[:,i] -= cj*t[:,j]
        t[:,i] /= np.linalg.norm(t[:,i])
    for i in range(n2):
        csi = t[i,:6]
        so = ' '.join(['%10.2f '%si for si in csi])
        print(aolbs[i], so)
    return t

def get_new_dm1(mol, mf, t):
    cs = mf.mo_coeff
    return cs

In [111]:
bst = 'cc-pvdz'
zs = [9,1]; coords = [[0.,0.,0],[0.,0.,0.98]]; m = ase.Atoms(zs,coords)
mol, _ = calc(m, bst=bst, icab=T)
#t = get_hao(mol)
mf = scf.RHF(mol)
mf.kernel()

-99.99358813512816

In [121]:
np.set_printoptions(precision=4,suppress=True)
s = mol.intor_symmetric('int1e_ovlp_sph')
p = reduce(np.dot, (s, mf.make_rdm1(), s))
import scipy
e,v = scipy.linalg.eigh(p[:-1,:-1], s[:-1,:-1])
eigs1 = e[::-1]; vs1 = v[:,::-1]
print(eigs1)
#o0, c0 = lo.nao._prenao_sub(mol, p, s)
#print(o0, c0)

[ 2.      2.      2.      2.      1.8886  0.      0.      0.     -0.
 -0.     -0.     -0.     -0.     -0.    ]


In [128]:
vs1[13]

array([ 0.,  0.,  0.,  0., -0.,  0., -0., -1.,  0., -0., -0., -0., -0.,
       -0.])

In [130]:
# write HAO
import interfaces._pyscf as pscf
reload(pscf)
oo = pscf.io(mol)
c1 = np.zeros((mol.nao,mol.nao))
c1[:-1,:-1] = vs1
orig, cell, dt = oo.orbital(c1, grids=[100,100,100], label='ch4-carbon')

In [95]:
mol.ao_labels()

['0 F 1s    ',
 '0 F 2s    ',
 '0 F 3s    ',
 '0 F 2px   ',
 '0 F 2py   ',
 '0 F 2pz   ',
 '0 F 3px   ',
 '0 F 3py   ',
 '0 F 3pz   ',
 '0 F 3dxy  ',
 '0 F 3dyz  ',
 '0 F 3dz^2 ',
 '0 F 3dxz  ',
 '0 F 3dx2-y2',
 '1 H 1s    ']

In [96]:
m.rotate(90, [1,1,1])
#a = calc_hao(m, bst=bst)
mol, _ = calc(m, bst=bst, icab=T)
#t = get_hao(mol)
mf = scf.RHF(mol)
mf.kernel()

-99.99358813502536

In [97]:
s = mol.intor_symmetric('int1e_ovlp_sph')
p = reduce(np.dot, (s, mf.make_rdm1(), s))
import scipy
e,v = scipy.linalg.eigh(p[:-1,:-1], s[:-1,:-1])
eigs2 = e[::-1]; vs2 = v[:,::-1]
print(eigs2)
#o0, c0 = lo.nao._prenao_sub(mol, p, s)
#print(o0, c0)

[ 2.    2.    2.    2.    1.89  0.    0.    0.    0.   -0.   -0.   -0.
 -0.   -0.  ]


In [103]:
np.all( np.abs(eigs2-eigs1)<1e-6 )

True

In [123]:
s = mol.intor_symmetric('int1e_ovlp_sph')
p = reduce(np.dot, (s, mf.make_rdm1(), s))
o0, c0 = lo.nao._prenao_sub(mol, p, s)
print(o0) #, c0)

[2.     1.9954 0.028  1.9409 1.9409 1.9409 0.0096 0.0096 0.0096 0.0027
 0.0027 0.0027 0.0027 0.0027 1.2034]


In [129]:
from mayavi import mlab
mlab.init_notebook()

ImportError: No module named ipyevents